In [53]:
from __future__ import division
import os
import numpy as np
import pandas as pd
from skimage.io import imread, imshow, imsave, show
from IPython.display import Image, HTML
pd.set_option('display.max_colwidth', -1)

## Load data

In [54]:
expected_tags = {}
all_tags = set(["agriculture","artisinal_mine","bare_ground","blooming","blow_down","clear","cloudy","conventional_mine","cultivation","habitation","haze","partly_cloudy","primary","road","selective_logging","slash_burn","water"])
with open("./rawInput/train.csv", "r") as train_labels:
    train_labels.readline()
    for line in train_labels:
        img, tags = line.rstrip("\n").split(",")
        expected_tags[img] = tags.split(" ")
print(len(expected_tags))

40479


In [55]:
predicted_tags = {}
with open("./predict/train-predict.csv", "r") as predict_labels:
    predict_labels.readline()
    for line in predict_labels:
        img, tags = line.rstrip("\n").split(",")
        predicted_tags[img] = tags.split(" ")
print(len(predicted_tags))

404


In [56]:
predicted_proba = {}
with open("./predict/train-predict-raw.csv", "r") as predict_proba:
    tags = predict_proba.readline().rstrip("\n").split(",")[1].split(" ")
    for line in predict_proba:
        img, proba = line.rstrip("\n").split(",")
        proba = proba.split(" ")
        predicted_proba[img] = { t: float(v) for t, v in zip(tags, proba) }
print(len(predicted_proba))

404


In [57]:
training_set = set([])
validation_set = set([])
with open("./train/training-files.csv", "r") as training_files:
    original_training_set = set(training_files.readline().strip().split(","))
    for f in os.listdir("./rawInput/train-jpg"):
        filename = f.split(".")[0]
        s = None
        if filename in original_training_set:
            s = training_set
        elif filename in predicted_tags:
            s = validation_set
        if s is not None:
            s.add(filename)
#         for i in range(8):
#             s.add("{name}--{i}".format(name=filename, i=i))

print("total images:", len(os.listdir("./rawInput/train-jpg")))
print("training set:", len(training_set))
print("validation set:", len(validation_set))

total images: 40479
training set: 40
validation set: 364


## Read predictions

### Per label score

In [58]:
# First value is number of correctly predicted
# Second is number of not detected (false negative)
# Third is number of inaccurately predicted (false positive)
tags_score = {t: { "tp":0, "fn": 0, "fp": 0} for t in all_tags}

def scores(tp, fp, fn):
    p = tp / (tp + fp + 1)
    r = tp / (tp + fn + 1)
    b = 2
    if p == 0 or r == 0:
        f2 = 0
    else:
        b = 2
        f2 = (1 + b**2) * p * r / (b**2*p + r)
    return p, r, f2
    
for img in validation_set:
    true_tags = expected_tags[img]
    pred_tags = predicted_tags[img]
    for t in true_tags:
        if t in pred_tags:
            tags_score[t]["tp"] += 1
        else:
            tags_score[t]["fn"] += 1
    for t in pred_tags:
        if t not in true_tags:
            tags_score[t]["fp"] += 1

data = []
for tag in all_tags:
    s = tags_score[tag]
    p, r, f2 = scores(s["tp"], s["fp"], s["fn"])
    data.append([tag, s["tp"], s["fp"], s["fn"], p, r, f2])
data.sort(key = lambda l:l[-1])
data = np.array(data)
columns = ["tp", "fp", "fn", "validation precision", "validation recall", "f2"]
df = pd.DataFrame(data=data[:, 1:], index=data[:, 0], columns=columns)
df

,tp,fp,fn,validation precision,validation recall,f2
haze,0,0,20,0.0,0.0,0
bare_ground,0,0,9,0.0,0.0,0
road,0,0,61,0.0,0.0,0
selective_logging,0,0,3,0.0,0.0,0
water,0,0,59,0.0,0.0,0
slash_burn,0,0,2,0.0,0.0,0
blooming,0,36,5,0.0,0.0,0
cultivation,0,0,46,0.0,0.0,0
conventional_mine,0,0,1,0.0,0.0,0
partly_cloudy,0,0,68,0.0,0.0,0


## F2 score

In [59]:
LABELS = ['clear', 'cloudy', 'haze', 'partly_cloudy', 'agriculture', 'artisinal_mine', 'bare_ground', 'blooming', 'blow_down', 'conventional_mine', 'cultivation', 'habitation', 'primary', 'road', 'selective_logging', 'slash_burn', 'water']

In [60]:
import numpy as np
from sklearn.metrics import fbeta_score
def F2Score(predicted, actual):
    # see https://www.kaggle.com/c/planet-understanding-the-amazon-from-space#evaluation
    predicted = set(predicted)
    actual = set(actual)
    tp = len(predicted & actual)
    tn = len(LABELS) - len(predicted | actual)
    fp = len(predicted) - tp
    fn = (len(LABELS) - len(predicted)) - tn
    return scores(tp, fp, fn)[2]

def scoreSet(data_set):
    print("scoreSet", len(data_set), len(expected_tags), len(predicted_tags))
    scores = []
    f2, n = 0, 0
    for img in data_set:
        real_tags = expected_tags[img]
        pred_tags = predicted_tags[img]
        s = F2Score(real_tags, pred_tags)
        f2 += s
        scores.append({ "img": img, "f2": s })
        n += 1
    return f2 / n, scores

In [61]:
training_score, training_scores = scoreSet(training_set)
validation_score, validation_scores = scoreSet(validation_set)
training_scores.sort(key = lambda l: l["f2"])
validation_scores.sort(key = lambda l: l["f2"])

print("F2 score for training set", training_score)
print("F2 score validation set", validation_score)

scoreSet 40 40479 404
scoreSet 364 40479 404
F2 score for training set 0.4578484015984018
F2 score validation set 0.4770546907647071


## Prediction per image

In [63]:
short_labels = {"agriculture": "ag","artisinal_mine": "a_mi","bare_ground": "b_gr","blooming": "bloom","blow_down": "blow","clear": "clea","cloudy": "clou","conventional_mine": "c_mi","cultivation": "cult","habitation": "hab","haze": "haze","partly_cloudy": "p_clo","primary": "prim","road": "road","selective_logging": "s_log","slash_burn": "sl_b","water": "wat"}

# return an HTML display for one prediction entry, making it green, red or grey
def getLabelPred(tag, img):
    truth = tag in expected_tags[img]
    pred = tag in predicted_tags[img]
    prob = predicted_proba[img][tag]
    isGood = truth == pred
    if truth == pred:
        cl = "green" if truth else "grey"
    else:
        cl = "red true" if truth else "red false"
    return '<span title="{tag}" class="{cl}">{val}</span>'.format(tag=tag, val=round(prob, 2), cl=cl)
        
# Return a dataframe the predictions
def getPrediction(data):
    img = data["img"]
    f2 = data["f2"]
    image = '<img src="./rawInput/train-jpg/{img}.jpg" class="input-image"/>'.format(img=img)
    
    return [
        img, 
        image, 
        f2
    ] + [getLabelPred(t, img) for t in all_tags]

table = []
originals = set([])
for data in validation_scores[:1000]:
    img = data["img"]
    original = img.split("--")[0]
    if original not in originals:
        table.append(getPrediction(data))
        originals.add(original)

table = np.array(table)

columns = ["image", "f2"] + [short_labels[t] for t in all_tags]
df = pd.DataFrame(data=table[:, 1:], index=table[:, 0], columns=columns)
style = "<style>.input-image{width: 50px; max-width: 250px !important;} .input-image:hover{width: 250px;} .green{color: green;} .grey{color: #e0e0e0;} .red{color: black;} .true{border-bottom: 3px solid green;} .false{border-bottom: 3px solid red;}</style>"
HTML(style + df.to_html(escape=False))

,image,f2,ag,haze,b_gr,road,s_log,wat,sl_b,bloom,clea,cult,c_mi,p_clo,a_mi,hab,prim,blow,clou
train_34737,,0,0.29,0.14,0.19,0.39,0.11,0.15,0.02,0.57,0.41,0.07,0.05,0.09,0.04,0.09,0.92,0.07,0.52
train_15915,,0,0.9,0.01,0.06,0.32,0.02,0.08,0.0,0.59,0.7,0.0,0.01,0.02,0.01,0.03,0.99,0.01,0.18
train_25192,,0,0.38,0.09,0.09,0.37,0.03,0.06,0.01,0.6,0.32,0.02,0.01,0.04,0.01,0.03,0.97,0.02,0.52
train_38278,,0,0.76,0.19,0.31,0.45,0.28,0.38,0.23,0.48,0.6,0.18,0.3,0.29,0.24,0.3,0.77,0.25,0.34
train_498,,0.21739130434782605,0.19,0.02,0.06,0.23,0.03,0.04,0.0,0.69,0.42,0.01,0.01,0.01,0.0,0.01,0.99,0.01,0.56
train_12227,,0.21739130434782608,1.0,0.0,0.0,0.18,0.0,0.0,0.0,0.61,0.97,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.01
train_6454,,0.21739130434782608,1.0,0.0,0.0,0.02,0.0,0.0,0.0,0.62,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
train_30107,,0.22727272727272727,0.12,0.09,0.12,0.29,0.12,0.14,0.01,0.6,0.31,0.06,0.03,0.03,0.02,0.05,0.95,0.06,0.66
train_12406,,0.22727272727272727,0.2,0.0,0.01,0.18,0.0,0.0,0.0,0.78,0.28,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.53
train_28481,,0.22727272727272727,0.32,0.02,0.07,0.28,0.04,0.06,0.0,0.69,0.48,0.01,0.01,0.01,0.01,0.02,0.98,0.02,0.52
